# Traindata 전처리

# JSON -> DataFrame

In [1]:
import os
import json
import pandas as pd

# 라벨링 JSON 파일이 있는 최상위 폴더 경로
label_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/라벨링데이터/"

# 실제 WAV 파일이 존재하는 원천 데이터의 최상위 경로
wav_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/1.Training/원천데이터/"

# 정상적으로 처리된 데이터 정보를 담을 리스트
data = []

# 오류 발생 시 해당 JSON 파일 또는 존재하지 않는 WAV 경로를 저장할 리스트
broken_files = []

# 라벨링 폴더 내부의 모든 JSON 파일을 재귀적으로 탐색
for folder_path, _, files in os.walk(label_root):
    for file_name in files:
        if file_name.endswith(".json"):
            # 현재 JSON 파일의 전체 경로 구성
            json_path = os.path.join(folder_path, file_name)
            try:
                # JSON 파일 열기 및 파싱
                with open(json_path, 'r', encoding='utf-8') as f:
                    content = json.load(f)

                # JSON 내부 정보 추출
                emotion = content["화자정보"]["Emotion"]
                style = content["화자정보"].get("SpeechStyle", "N/A")
                sensitivity = content["화자정보"].get("Sensitivity", "N/A")
                wav_file = content["파일정보"]["FileName"]

                # 현재 JSON 경로를 라벨 기준 상대경로로 변환
                relative_path = os.path.relpath(folder_path, start=label_root)

                # 상대 경로에서 모든 TL을 TS로 변경
                relative_path = relative_path.replace("TL", "TS")

                # WAV 경로를 원천 데이터 기준으로 재구성
                wav_path = os.path.join(wav_root, relative_path, wav_file)

                # WAV 파일 존재 여부 확인
                if os.path.exists(wav_path):
                    # 정상 데이터 추가
                    data.append({
                        "wav_path": wav_path,
                        "emotion": emotion,
                        "style": style,
                        "sensitivity": sensitivity
                    })
                else:
                    # WAV 파일이 존재하지 않는 경우 로그에 기록
                    print(f"WAV 파일 없음: {wav_path}")
                    broken_files.append(wav_path)

            except Exception as e:
                # JSON 파싱 중 오류 발생 시 기록
                print(f"JSON 읽기 오류: {json_path}: {e}")
                broken_files.append(json_path)

# 정상적으로 수집된 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# 결과 CSV 파일로 저장
os.makedirs("./data/usou", exist_ok=True)
df.to_csv("./data/usou/metadata_cleaned.csv", index=False)

# 오류가 발생한 경로들을 텍스트 파일로 저장
with open("./data/usou/broken_files.txt", "w") as f:
    for path in broken_files:
        f.write(path + "\n")

# 최종 처리 결과 출력
print(f"정상 처리된 JSON 수: {len(df)}")
print(f"에러 발생 수: {len(broken_files)}")


정상 처리된 JSON 수: 815491
에러 발생 수: 0


# MFCC 추출
## MFCC 추출이란
- 음성에서 특징을 뽑아낸 백터
## 데이터 형태
- 2차원 배열(시간 프레임수, 13)
# 배치
- 배치 : 전체 데이터를 나누어 처리하는 단위
## 나누는 이유
- 메모리 부족으로 컴퓨터 프리징 발생





In [1]:
import os
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm

# ============================
# 1. 메타데이터 로드
# ============================
# 사전에 정제된 메타데이터 CSV 파일 경로
csv_path = "/media/usou/PortableSSD/mldl_project/data/metadata_cleaned.csv"
df = pd.read_csv(csv_path)

# ============================
# 2. 설정값 정의
# ============================
sample_rate = 16000            # 음성 파일 샘플링 레이트 (Hz)
max_duration = 5.0             # WAV 파일 최대 로딩 시간 (초) → 너무 긴 파일 방지
save_interval = 10000          # 몇 개마다 배치로 저장할지 설정

# 저장용 리스트 초기화
mfcc_features = []             # MFCC 벡터 리스트
labels = []                    # 감정 레이블 리스트
error_files = []               # 처리 중 실패한 파일 목록
save_counter = 0               # 배치 저장 인덱스

# 저장 디렉토리 설정
save_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
os.makedirs(save_dir, exist_ok=True)

# ============================
# 3. MFCC 추출 루프
# ============================
for idx, row in tqdm(df.iterrows(), total=len(df)):
    wav_path = row["wav_path"]  # 메타데이터에 포함된 wav 파일 전체 경로
    try:
        # WAV 파일 로딩 (최대 max_duration 초까지만 로드)
        y, sr = librosa.load(wav_path, sr=sample_rate, duration=max_duration)

        # MFCC 13차원 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

        # 시간 축 기준으로 전치 (time_step, n_mfcc)
        mfcc_features.append(mfcc.T)
        labels.append(row["emotion"])

    except Exception as e:
        # 에러 발생 시 파일 경로 저장
        print(f"Error processing {wav_path}: {e}")
        error_files.append(wav_path)

    # 일정 수 이상 쌓이면 배치 저장 후 메모리 초기화
    if len(mfcc_features) >= save_interval:
        np.save(os.path.join(save_dir, f"mfcc_batch_{save_counter}.npy"), np.array(mfcc_features, dtype=object))
        np.save(os.path.join(save_dir, f"label_batch_{save_counter}.npy"), np.array(labels))
        save_counter += 1
        mfcc_features = []
        labels = []

# 남은 데이터가 있다면 마지막 배치 저장
if mfcc_features:
    np.save(os.path.join(save_dir, f"mfcc_batch_{save_counter}.npy"), np.array(mfcc_features, dtype=object))
    np.save(os.path.join(save_dir, f"label_batch_{save_counter}.npy"), np.array(labels))

# ============================
# 4. 에러 파일 저장
# ============================
error_log_path = "/media/usou/PortableSSD/mldl_project/data/broken_audio_files.txt"
with open(error_log_path, "w") as f:
    for path in error_files:
        f.write(path + "\n")

# ============================
# 5. 처리 결과 출력
# ============================
print(f"성공적으로 저장된 배치 수: {save_counter + 1}")
print(f"실패한 파일 수: {len(error_files)}")


/tmp/ipykernel_17508/736413005.py:12: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
100%|██████████| 815491/815491 [2:58:19<00:00, 76.22it/s]   


성공적으로 저장된 배치 수: 82
실패한 파일 수: 0


# 레이블 인코딩

In [ ]:
import os
import numpy as np
import glob
import pickle
from sklearn.preprocessing import LabelEncoder

# ============================
# 1. 설정
# ============================
# 레이블 배치가 저장된 경로
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"

# 인코딩된 레이블 저장 경로
encoded_label_dir = os.path.join(label_dir, "encoded_labels")
os.makedirs(encoded_label_dir, exist_ok=True)

# ============================
# 2. 모든 배치 레이블 수집
# ============================
# label_batch_*.npy 파일 경로 리스트
label_files = sorted(glob.glob(os.path.join(label_dir, "label_batch_*.npy")))

# 전체 레이블 리스트 생성
all_labels = []
batch_label_data = []  # 배치별 데이터도 임시 저장
for label_file in label_files:
    labels = np.load(label_file, allow_pickle=True)
    batch_label_data.append(labels)
    all_labels.extend(labels)

# ============================
# 3. 레이블 인코딩
# ============================
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# 인코더 저장 (추후 예측 결과 복원용)
with open(os.path.join(label_dir, "label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

# ============================
# 4. 인코딩된 레이블 배치별로 저장
# ============================
for i, labels in enumerate(batch_label_data):
    encoded = label_encoder.transform(labels)
    save_path = os.path.join(encoded_label_dir, f"label_batch_{i}.npy")
    np.save(save_path, encoded)

print(f"총 레이블 개수: {len(all_labels)}")
print(f"인코딩된 클래스 목록: {label_encoder.classes_}")
print(f"배치 수: {len(label_files)}")
print("레이블 인코딩 및 저장 완료")


총 레이블 개수: 815491
인코딩된 클래스 목록: ['Angry' 'Anxious' 'Embarrassed' 'Happy' 'Hurt' 'Neutrality' 'Sad' 'nan']
배치 수: 82
✅ 레이블 인코딩 및 저장 완료


# CNN 모델 정의

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape, num_classes):
    """
    CNN 기반 음성 감정 분류 모델 정의

    Parameters:
        input_shape (tuple): 입력 데이터 형태 (예: (시간축 길이, MFCC 차원 수, 채널 수))
        num_classes (int): 분류할 감정 클래스 수

    Returns:
        tensorflow.keras.Model: 컴파일 완료된 CNN 모델
    """
    model = models.Sequential()

    # 첫 번째 컨볼루션 레이어: 필터 수 32, 커널 사이즈 3x3, 활성화 함수 ReLU
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    # 배치 정규화: 학습 안정성과 속도 개선
    model.add(layers.BatchNormalization())
    # 최대 풀링: 출력 크기 절반으로 줄임 (특징 추출과 과적합 방지)
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # 두 번째 컨볼루션 레이어: 필터 수 64
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # 세 번째 컨볼루션 레이어: 필터 수 128
    model.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    # 전역 평균 풀링: 전체 피처 맵의 평균을 계산하여 1D 벡터로 변환
    model.add(layers.GlobalAveragePooling2D())

    # 완전 연결층(Dense Layer) 추가
    model.add(layers.Dense(128, activation='relu'))
    # 과적합 방지를 위한 드롭아웃 (30%)
    model.add(layers.Dropout(0.3))
    # 출력층: softmax로 감정 클래스 확률 예측
    model.add(layers.Dense(num_classes, activation='softmax'))

    # 모델 컴파일: Adam 옵티마이저, sparse_categorical_crossentropy 손실 함수, 정확도 지표
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


2025-03-27 14:35:02.667534: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743053702.684248   21141 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743053702.688507   21141 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743053702.700707   21141 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743053702.700724   21141 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743053702.700725   21141 computation_placer.cc:177] computation placer alr

# Validation 전처리

# 메타데이터 csv로 저장

In [5]:
import os
import json
import pandas as pd

# ========================================
# 1. 경로 설정
# ========================================

# 라벨링 JSON 파일이 저장된 루트 폴더
label_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/라벨링데이터/VL1"

# 실제 음성 WAV 파일이 있는 루트 폴더
wav_root = "/media/usou/PortableSSD/mldl/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/VS1"

# ========================================
# 2. 결과 저장 리스트 초기화
# ========================================
data = []             # 메타데이터 저장용 리스트
broken_files = []     # 에러 발생한 파일 로그용 리스트

# ========================================
# 3. JSON 파일 순회 및 정보 추출
# ========================================
for folder_path, _, files in os.walk(label_root):
    for file_name in files:
        if file_name.endswith(".json"):
            json_path = os.path.join(folder_path, file_name)
            try:
                # JSON 파일 열기
                with open(json_path, 'r', encoding='utf-8') as f:
                    content = json.load(f)

                # 화자 정보에서 감정, 스타일, 세부 감정 추출
                emotion = content["화자정보"]["Emotion"]
                style = content["화자정보"].get("SpeechStyle", "N/A")
                sensitivity = content["화자정보"].get("Sensitivity", "N/A")

                # WAV 파일 이름 추출
                wav_file = content["파일정보"]["FileName"]

                # 현재 JSON 경로에서 라벨 루트를 기준으로 상대 경로 추출
                relative_path = os.path.relpath(folder_path, start=label_root)

                # 실제 WAV 파일 경로 생성
                wav_path = os.path.join(wav_root, relative_path, wav_file)

                # WAV 파일이 존재하면 메타데이터에 추가
                if os.path.exists(wav_path):
                    data.append({
                        "wav_path": wav_path,
                        "emotion": emotion,
                        "style": style,
                        "sensitivity": sensitivity
                    })
                else:
                    # WAV 파일이 없는 경우 기록
                    print(f"WAV 파일 없음: {wav_path}")
                    broken_files.append(wav_path)

            except Exception as e:
                # JSON 파싱 실패 시 기록
                print(f"JSON 읽기 오류: {json_path}, 에러: {e}")
                broken_files.append(json_path)

# ========================================
# 4. 결과 저장
# ========================================

# DataFrame 생성
df = pd.DataFrame(data)

# 저장 경로 생성
os.makedirs("/media/usou/PortableSSD/mldl_project/data/validation", exist_ok=True)

# 메타데이터 CSV 저장
df.to_csv("/media/usou/PortableSSD/mldl_project/data/validation/metadata_cleaned_val.csv", index=False)

# 에러 파일 로그 저장
with open("/media/usou/PortableSSD/mldl_project/data/validation/broken_val_files.txt", "w") as f:
    for path in broken_files:
        f.write(path + "\n")

# 요약 출력
print(f"정상 처리된 JSON 수: {len(df)}")
print(f"에러 발생 수: {len(broken_files)}")


정상 처리된 JSON 수: 112157
에러 발생 수: 0


# MFCC 추출 Validation 용

In [6]:
import os
import librosa
import pandas as pd
import numpy as np
from tqdm import tqdm

# ========================================
# 1. 메타데이터 로드
# ========================================

# validation용 정제된 메타데이터 CSV 경로
csv_path = "/media/usou/PortableSSD/mldl_project/data/validation/metadata_cleaned_val.csv"
df = pd.read_csv(csv_path)

# ========================================
# 2. 설정값 정의
# ========================================

sample_rate = 16000             # 음성 샘플링 레이트 (16kHz)
max_duration = 5.0              # WAV 최대 로딩 시간 (초)
save_interval = 10000           # 배치 저장 기준 개수
save_dir = "/media/usou/PortableSSD/mldl_project/data/validation/mfcc_batches"
os.makedirs(save_dir, exist_ok=True)

# 저장용 리스트 초기화
mfcc_features = []              # 추출된 MFCC 벡터 리스트
labels = []                     # 감정 레이블 리스트
error_files = []                # 실패한 파일 목록
save_counter = 0                # 배치 파일 번호

# ========================================
# 3. MFCC 추출 루프
# ========================================

for idx, row in tqdm(df.iterrows(), total=len(df)):
    wav_path = row["wav_path"]

    try:
        # WAV 파일 로딩 (최대 max_duration 초까지만 로드)
        y, sr = librosa.load(wav_path, sr=sample_rate, duration=max_duration)

        # MFCC 추출 (13차원)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

        # 시간 축 기준으로 전치 (time_step, 13)
        mfcc_features.append(mfcc.T)
        labels.append(row["emotion"])

    except Exception as e:
        # 로딩 실패 시 에러 출력 및 로그 저장
        print(f"Error processing {wav_path}: {e}")
        error_files.append(wav_path)

    # 일정 개수 이상이면 배치 저장
    if len(mfcc_features) >= save_interval:
        np.save(os.path.join(save_dir, f"mfcc_batch_{save_counter}.npy"), np.array(mfcc_features, dtype=object))
        np.save(os.path.join(save_dir, f"label_batch_{save_counter}.npy"), np.array(labels))
        save_counter += 1
        mfcc_features = []
        labels = []

# 루프 종료 후 남은 데이터 저장
if mfcc_features:
    np.save(os.path.join(save_dir, f"mfcc_batch_{save_counter}.npy"), np.array(mfcc_features, dtype=object))
    np.save(os.path.join(save_dir, f"label_batch_{save_counter}.npy"), np.array(labels))

# ========================================
# 4. 에러 파일 저장
# ========================================

error_log_path = "/media/usou/PortableSSD/mldl_project/data/validation/broken_audio_files_val.txt"
with open(error_log_path, "w") as f:
    for path in error_files:
        f.write(path + "\n")

# ========================================
# 5. 처리 결과 출력
# ========================================

print(f"성공적으로 저장된 배치 수: {save_counter + 1}")
print(f"실패한 파일 수: {len(error_files)}")


100%|██████████| 112157/112157 [27:27<00:00, 68.06it/s]

성공적으로 저장된 배치 수: 12
실패한 파일 수: 0


# Validation 데이터용 레이블 인코딩

In [7]:
import os
import numpy as np
import glob
import pickle
from sklearn.preprocessing import LabelEncoder

# ============================
# 1. 설정
# ============================
# 레이블 배치가 저장된 경로
label_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"

# 인코딩된 레이블 저장 경로
encoded_label_dir = os.path.join(label_dir, "encoded_labels")
os.makedirs(encoded_label_dir, exist_ok=True)

# ============================
# 2. 모든 배치 레이블 수집
# ============================
# label_batch_*.npy 파일 경로 리스트
label_files = sorted(glob.glob(os.path.join(label_dir, "label_batch_*.npy")))

# 전체 레이블 리스트 생성
all_labels = []
batch_label_data = []  # 배치별 데이터도 임시 저장
for label_file in label_files:
    labels = np.load(label_file, allow_pickle=True)
    batch_label_data.append(labels)
    all_labels.extend(labels)

# ============================
# 3. 레이블 인코딩
# ============================
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# 인코더 저장 (추후 예측 결과 복원용)
with open(os.path.join(label_dir, "label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

# ============================
# 4. 인코딩된 레이블 배치별로 저장
# ============================
for i, labels in enumerate(batch_label_data):
    encoded = label_encoder.transform(labels)
    save_path = os.path.join(encoded_label_dir, f"label_batch_{i}.npy")
    np.save(save_path, encoded)

print(f"총 레이블 개수: {len(all_labels)}")
print(f"인코딩된 클래스 목록: {label_encoder.classes_}")
print(f"배치 수: {len(label_files)}")
print("레이블 인코딩 및 저장 완료")


총 레이블 개수: 815491
인코딩된 클래스 목록: ['Angry' 'Anxious' 'Embarrassed' 'Happy' 'Hurt' 'Neutrality' 'Sad' 'nan']
배치 수: 82
레이블 인코딩 및 저장 완료


# 학습!

In [ ]:
import os
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ========================
# 1. 배치 로딩 함수 정의
# ========================
def load_batches(batch_dir, prefix):
    mfcc_list = []
    label_list = []
    i = 0
    while True:
        mfcc_path = os.path.join(batch_dir, f"{prefix}_batch_{i}.npy")
        label_path = os.path.join(batch_dir, "encoded_labels", f"label_batch_{i}.npy")
        if not os.path.exists(mfcc_path):
            break
        mfccs = np.load(mfcc_path, allow_pickle=True)
        labels = np.load(label_path)
        mfcc_list.extend(mfccs)
        label_list.extend(labels)
        i += 1
    return mfcc_list, label_list

# ========================
# 2. 학습 / 검증 데이터 로딩
# ========================
train_dir = "/media/usou/PortableSSD/mldl_project/data/mfcc_batches"
val_dir = "/media/usou/PortableSSD/mldl_project/data/validation_batches"

X_train_raw, y_train = load_batches(train_dir, "mfcc")
X_val_raw, y_val = load_batches(val_dir, "mfcc")

# ========================
# 3. 패딩 및 형태 변환
# ========================
max_len = max([x.shape[0] for x in X_train_raw + X_val_raw])
X_train = pad_sequences(X_train_raw, maxlen=max_len, dtype='float32', padding='post')
X_val = pad_sequences(X_val_raw, maxlen=max_len, dtype='float32', padding='post')

# CNN 입력을 위해 채널 차원 추가
X_train = np.expand_dims(X_train, -1)
X_val = np.expand_dims(X_val, -1)

y_train = np.array(y_train)
y_val = np.array(y_val)

# ========================
# 4. CNN 모델 정의
# ========================
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalAveragePooling2D())

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# ========================
# 5. 모델 생성 및 학습
# ========================
input_shape = X_train.shape[1:]  # (time, n_mfcc, 1)
num_classes = len(np.unique(y_train))

model = create_cnn_model(input_shape, num_classes)

# 콜백 설정
callbacks = [
    ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy', mode='max')
]

# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)


: 